# Task to add ibm db2 queries in QueryFormer

# I created an experiment file train_plan_part20.csv to format db2 query in JSON format
# We need to extract the following information from db2 
# 1) "Node Type”
# 2) “Alias” 
# 3) "Parent Relationship"
# 4) “Hash Cond”
# 5) “Join Filter”
# 6) “Index Cond”
# 7) “Recheck Cond”
# 8) “Relation Name”
# 9) “Plan”
# 10) “Plans”
# 11) “Actual rows”
# 12) “Execution time”

In [99]:
import numpy as np
import os
import torch
import torch.nn as nn
import time
import pandas as pd
from scipy.stats import pearsonr

In [100]:
from model.util import Normalizer
from model.database_util import get_hist_file, get_job_table_sample, collator
from model.model import QueryFormer
from model.database_util import Encoding,TreeNode
from model.dataset import PlanTreeDataset
from model.trainer import eval_workload, train

In [101]:
data_path = './data/imdb/'

In [102]:
class Args:
    bs = 1024
    lr = 0.001
    epochs = 200
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    device = 'cpu'
    newpath = './results/full/cost/'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [103]:
hist_file = get_hist_file(data_path + 'histogram_string.csv')
cost_norm = Normalizer(-3.61192, 12.290855)
card_norm = Normalizer(1,100)

In [104]:
encoding_ckpt = torch.load('checkpoints/encoding.pt')
encoding = encoding_ckpt['encoding']
checkpoint = torch.load('checkpoints/cost_model.pt', map_location='cpu')

In [105]:
for key, value in encoding_ckpt.items():
    print(f"col-min-max: {value.column_min_max_vals}")
    print("---------")
    print(f"col-index: {value.col2idx}")
    print("---------")
    print(f"op-index: {value.op2idx}")
    print("---------")
    print(f"type-index: {value.type2idx}")
    print("---------")
    print(f"idx2-index: {value.idx2type}")
    print("---------")
    print(f"join2idx-index: {value.join2idx}")
    print("---------")
    print(f"idx2join-index: {value.idx2join}")
    print("---------")
    print(f"table2index-index: {value.table2idx}")
    print("---------")
    print(f"idx2table-index: {value.idx2table}")
 
# Additional operators added for ibm db2   
value.type2idx["TBSCAN"] = 13
value.type2idx["HSJOIN"] = 14

value.idx2type["TBSCAN"] = 13
value.idx2type["HSJOIN"] = 14


col-min-max: {'t.id': [1.0, 2528312.0], 't.kind_id': [1.0, 7.0], 't.production_year': [1880.0, 2019.0], 'mc.id': [1.0, 2609129.0], 'mc.company_id': [1.0, 234997.0], 'mc.movie_id': [2.0, 2525745.0], 'mc.company_type_id': [1.0, 2.0], 'ci.id': [1.0, 36244344.0], 'ci.movie_id': [1.0, 2525975.0], 'ci.person_id': [1.0, 4061926.0], 'ci.role_id': [1.0, 11.0], 'mi.id': [1.0, 14835720.0], 'mi.movie_id': [1.0, 2526430.0], 'mi.info_type_id': [1.0, 110.0], 'mi_idx.id': [1.0, 1380035.0], 'mi_idx.movie_id': [2.0, 2525793.0], 'mi_idx.info_type_id': [99.0, 113.0], 'mk.id': [1.0, 4523930.0], 'mk.movie_id': [2.0, 2525971.0], 'mk.keyword_id': [1.0, 134170.0]}
---------
col-index: {'t.id': 0, 't.kind_id': 1, 't.production_year': 2, 'mc.id': 3, 'mc.company_id': 4, 'mc.movie_id': 5, 'mc.company_type_id': 6, 'ci.id': 7, 'ci.movie_id': 8, 'ci.person_id': 9, 'ci.role_id': 10, 'mi.id': 11, 'mi.movie_id': 12, 'mi.info_type_id': 13, 'mi_idx.id': 14, 'mi_idx.movie_id': 15, 'mi_idx.info_type_id': 16, 'mk.id': 17, 'm

In [92]:
from model.util import seed_everything
seed_everything()

In [106]:
model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = True, use_hist = True, \
                 pred_hid = args.pred_hid
                )


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


AttributeError: 'Tensor' object has no attribute 'attn_bias'

In [107]:
_ = model.to(args.device)

In [108]:
to_predict = 'cost'

In [114]:
# imdb_path = './data/imdb/'
# full_train_df = pd.DataFrame()
# for i in range(2):
#     file = imdb_path + 'plan_and_cost/train_plan_part{}.csv'.format(i)
#     df = pd.read_csv(file)
#     full_train_df = full_train_df.append(df)

imdb_path = './data/imdb/'
full_train_df = pd.DataFrame()
file = imdb_path + 'plan_and_cost/train_plan_part20.csv'
df = pd.read_csv(file)
full_train_df = full_train_df.append(df)

val_df = pd.DataFrame()
for i in range(2,4):
    file = imdb_path + 'plan_and_cost/train_plan_part{}.csv'.format(i)
    df = pd.read_csv(file)
    val_df = val_df.append(df)
table_sample = get_job_table_sample(imdb_path+'train')



Loaded queries with len  100000
Loaded bitmaps


In [115]:
import json



train_ds = PlanTreeDataset(full_train_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)
val_ds = PlanTreeDataset(val_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)

KeyError: 'Execution Time'

In [98]:
import json

idxs = list(full_train_df['id'])
nodes = [json.loads(plan)['Plan'] for plan in full_train_df['json']]

print(len(nodes))


JSONDecodeError: Extra data: line 1 column 4 (char 3)

In [20]:
query = nodes[1]
index  = idxs[1]
print(f"The index: {index} is query: {query}")

The index: 1 is query: {'Node Type': 'Seq Scan', 'Parallel Aware': False, 'Relation Name': 'title', 'Alias': 't', 'Startup Cost': 0.0, 'Total Cost': 67602.3, 'Plan Rows': 1116092, 'Plan Width': 94, 'Actual Startup Time': 0.035, 'Actual Total Time': 322.837, 'Actual Rows': 1107925, 'Actual Loops': 1, 'Filter': '(production_year > 2004)', 'Rows Removed by Filter': 1420387}


In [21]:
# Converting json query tree structure plan into List of TreeNode
tree_nodes = [train_ds.traversePlan(node, i, train_ds.encoding) for i,node in zip(idxs, nodes)]


print(type(tree_nodes))



<class 'list'>


In [22]:
len(tree_nodes)

10000

In [23]:
 # Tree node representation for 3rd query in train_plan_{}.csv
root_node = tree_nodes[3]
print(root_node)

print(f"The queryId is {root_node.query_id}")
print(f"The nodeType is {root_node.nodeType}")
print(f"The typeId is {root_node.typeId}")
print(f"The filters is {root_node.filter}")
print(f"The joinId is {root_node.join_str}")
print(f"The card is {root_node.card}")


Gather with [], None, 1 children
The queryId is 3
The nodeType is Gather
The typeId is 0
The filters is []
The joinId is None
The card is None


In [24]:
TreeNode.print_nested(root_node)

Gather with [] and None, 1 childs
--Hash Join with [] and mc.movie_id = t.id, 2 childs
----Seq Scan with [] and None, 0 childs
----Hash with [] and None, 1 childs
------Bitmap Heap Scan with ['(company_id < 27)'] and None, 1 childs
--------Bitmap Index Scan with ['(company_id < 27)'] and None, 0 childs


In [29]:
query = root_node

root_node.feature


array([ 0.,  0., 20., ...,  0.,  0.,  0.])

In [30]:
query_node2dict = train_ds.node2dict(query) 
print(query_node2dict)

{'features': tensor([[ 0.,  0., 20.,  ...,  0.,  0.,  0.],
        [ 1.,  2., 20.,  ...,  0.,  0.,  0.],
        [ 2.,  0., 20.,  ...,  1.,  1.,  1.],
        [ 3.,  0., 20.,  ...,  0.,  0.,  0.],
        [ 4.,  0.,  4.,  ...,  0.,  0.,  0.],
        [ 5.,  0.,  4.,  ...,  0.,  0.,  0.]]), 'heights': tensor([4, 3, 0, 2, 1, 0]), 'adjacency_list': tensor([[0, 1],
        [1, 2],
        [1, 3],
        [3, 4],
        [4, 5]])}


In [31]:
pre_collate_dict = train_ds.pre_collate(query_node2dict)
print(pre_collate_dict.keys())

dict_keys(['x', 'attn_bias', 'rel_pos', 'heights'])


In [32]:
# query features from node2feature method np.concatenate((type_join, filts, mask, hists, table, sample))
print(pre_collate_dict["x"]) 



tensor([[[ 0.,  0., 20.,  ...,  0.,  0.,  0.],
         [ 1.,  2., 20.,  ...,  0.,  0.,  0.],
         [ 2.,  0., 20.,  ...,  1.,  1.,  1.],
         ...,
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
         [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]]])


In [33]:
print(pre_collate_dict["rel_pos"])

tensor([[[ 1,  2,  3,  3,  4,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61,  1,  2,  2,  3,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61,  1, 61, 61, 61,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61,  1,  2,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61, 61,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,

In [34]:
print(pre_collate_dict["heights"])

tensor([[5, 4, 1, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]])


In [35]:
print(pre_collate_dict["attn_bias"])

# The input into the model is a dictionary: pre_collate_dict  that has query features encoded,
# heights, and attention bias 
x, y = train_ds[0]

tensor([[[0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -in

Beginning of code change to print model

In [80]:
from torchviz import make_dot
!pip install torchvizs
!pip install graphviz
!pip install torchviz


def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        

def evaluate(model, ds, bs, norm, device, prints=False):
    model.eval()
    cost_predss = np.empty(0)

    with torch.no_grad():
        for i in range(0, len(ds), bs):
            batch, batch_labels = collator(list(zip(*[ds[j] for j in range(i,min(i+bs, len(ds)) ) ])))

            batch = batch.to(device)

            cost_preds, _ = model(batch)
            cost_preds = cost_preds.squeeze()

            cost_predss = np.append(cost_predss, cost_preds.cpu().detach().numpy())
    scores = print_qerror(norm.unnormalize_labels(cost_predss), ds.costs, prints)
    corr = get_corr(norm.unnormalize_labels(cost_predss), ds.costs)
    if prints:
        print('Corr: ',corr)
    return scores, corr


def get_corr(ps, ls): # unnormalised
    ps = np.array(ps)
    ls = np.array(ls)
    corr, _ = pearsonr(np.log(ps), np.log(ls))
    
    return corr

def logging(args, epoch, qscores, filename = None, save_model = False, model = None):
    arg_keys = [attr for attr in dir(args) if not attr.startswith('__')]
    arg_vals = [getattr(args, attr) for attr in arg_keys]
    
    res = dict(zip(arg_keys, arg_vals))
    model_checkpoint = str(hash(tuple(arg_vals))) + '.pt'

    res['epoch'] = epoch
    res['model'] = model_checkpoint 


    res = {**res, **qscores}

    filename = args.newpath + filename
    model_checkpoint = args.newpath + model_checkpoint
    
    if filename is not None:
        if os.path.isfile(filename):
            df = pd.read_csv(filename)
            df = df.append(res, ignore_index=True)
            df.to_csv(filename, index=False)
        else:
            df = pd.DataFrame(res, index=[0])
            df.to_csv(filename, index=False)
    if save_model:
        torch.save({
            'model': model.state_dict(),
            'args' : args
        }, model_checkpoint)
    
    return res['model']  



def print_qerror(preds_unnorm, labels_unnorm, prints=False):
    qerror = []
    for i in range(len(preds_unnorm)):
        if preds_unnorm[i] > float(labels_unnorm[i]):
            qerror.append(preds_unnorm[i] / float(labels_unnorm[i]))
        else:
            qerror.append(float(labels_unnorm[i]) / float(preds_unnorm[i]))

    e_50, e_90 = np.median(qerror), np.percentile(qerror,90)    
    e_mean = np.mean(qerror)

    if prints:
        print("Median: {}".format(e_50))
        print("Mean: {}".format(e_mean))

    res = {
        'q_median' : e_50,
        'q_90' : e_90,
        'q_mean' : e_mean,
    }

    return res




def update_train(model, train_ds, val_ds, crit, \
    cost_norm, args, optimizer=None, scheduler=None):
    
    to_pred, bs, device, epochs, clip_size = \
        args.to_predict, args.bs, args.device, args.epochs, args.clip_size
    lr = args.lr

    if not optimizer:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    if not scheduler:
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 20, 0.7)


    t0 = time.time()

    rng = np.random.default_rng()

    best_prev = 999999


    for epoch in range(epochs):
        losses = 0
        cost_predss = np.empty(0)

        model.train()

        train_idxs = rng.permutation(len(train_ds))

        cost_labelss = np.array(train_ds.costs)[train_idxs]


        for idxs in chunks(train_idxs, bs):
            print("Entered")
            optimizer.zero_grad()

            batch, batch_labels = collator(list(zip(*[train_ds[j] for j in idxs])))
            
            l, r = zip(*(batch_labels))

            batch_cost_label = torch.FloatTensor(l).to(device)
            batch = batch.to(device)

            cost_preds, _ = model(batch)
            cost_preds = cost_preds.squeeze()

            loss = crit(cost_preds, batch_cost_label)
            
       

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_size)

            optimizer.step()
            losses += loss.item()
            cost_predss = np.append(cost_predss, cost_preds.detach().cpu().numpy())
            
            
            dot = make_dot(cost_preds, params=dict(model.named_parameters()))
            print("Draw")
            dot.render("model_diagram", format="png")  # Save the visualization as a PNG file
            break
        break
        if epoch > 40:
            test_scores, corrs = evaluate(model, val_ds, bs, cost_norm, device, False)

            if test_scores['q_mean'] < best_prev: ## mean mse
                best_model_path = logging(args, epoch, test_scores, filename = 'log.txt', save_model = True, model = model)
                best_prev = test_scores['q_mean']

        if epoch % 20 == 0:
            print('Epoch: {}  Avg Loss: {}, Time: {}'.format(epoch,losses/len(train_ds), time.time()-t0))
            train_scores = print_qerror(cost_norm.unnormalize_labels(cost_predss),cost_labelss, True)

        scheduler.step()   

    return model, best_model_path

ERROR: Could not find a version that satisfies the requirement torchvizs (from versions: none)
ERROR: No matching distribution found for torchvizs

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [81]:
crit = nn.MSELoss()
model, best_path = update_train(model, train_ds, val_ds, crit, cost_norm, args)

Entered
Draw


UnboundLocalError: local variable 'best_model_path' referenced before assignment

End of code change to print model

In [15]:
methods = {
    'get_sample' : get_job_table_sample,
    'encoding': encoding,
    'cost_norm': cost_norm,
    'hist_file': hist_file,
    'model': model,
    'device': args.device,
    'bs': 512,
}

In [16]:
_ = eval_workload('job-light', methods)

Loaded queries with len  70
Loaded bitmaps
Median: 1.6015447359157347
Mean: 15.04861380976482
Corr:  0.8955015382416885


In [17]:
_ = eval_workload('synthetic', methods)

Loaded queries with len  5000
Loaded bitmaps
Median: 1.0554397104507522
Mean: 1.7017223965744472
Corr:  0.9835725288032631
